In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import re
import datetime as dt
import random
import pickle
import shutil
import subprocess

# opencv
import cv2

# tensorflow/keras
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, Conv2D, Dropout, Lambda, Concatenate,Flatten, concatenate, LSTM, MaxPooling2D, Permute, Reshape, TimeDistributed
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.utils.data_utils import Sequence
from tensorflow.keras import Model
from tensorflow.keras.callbacks import EarlyStopping

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import scale
from sklearn.preprocessing import MinMaxScaler

from utilities import extract_position_time, normalize_datetime, normalize_y, get_lat_long_bounds

In [15]:
files_train = os.listdir('../images/train')
files_train = [f for f in files_train if f.endswith('.png')]

files_val = os.listdir('../images/valid')
files_val= [f for f in files_val if f.endswith('.png')]

files = files_train + files_val

# create array of position
position = np.array([extract_position_time(f)[0] for f in files])

# create array of time
times = np.array([extract_position_time(f)[1] for f in files])

In [4]:
def haversine_loss(y_true, y_pred, R=3443.92):
    # Convert normalized lat and long into actuals
    lat_min, lat_range, long_min, long_range = get_lat_long_bounds(position)
    lat1  = y_true[:,0] * lat_range + lat_min
    lat2  = y_pred[:,0] * lat_range + lat_min
    long1 = y_true[:,1] * long_range + long_min
    long2 = y_pred[:,1] * long_range + long_min 
    
    # Compute phis and lambdas 
    phi1 = lat1 * np.pi / 180
    phi2 = lat2 * np.pi / 180
    delta_phi    = (lat2 - lat1) * np.pi / 180
    delta_lambda = (long2 - long1) * np.pi / 180
    
    # Intermediate computations
    a = tf.square(tf.sin(delta_phi / 2)) + tf.cos(phi1) * tf.cos(phi2) * tf.square(tf.sin(delta_lambda / 2))
    c = 2 * tf.atan2(tf.sqrt(a), tf.sqrt(1 - a))
    
    # Compute distances
    d = R * c
    
    # Compute the mean squared distance (MSE)
    return tf.reduce_mean(d)

In [5]:
class CustomGenerator(Sequence):
    def __init__(self, directory, batch_size=32, segment = 'Train'):
        
        self.directory = directory
        self.batch_size = batch_size
        self.segment = segment

    def __len__(self):
      if self.segment !='Train':
        return int(np.floor(len(files_val) / self.batch_size))
      else:
        return int(np.floor(len(files_train) / self.batch_size))


    def __getitem__(self, idx):

        # get batch of images
        batch = os.listdir(self.directory)[idx * self.batch_size:(idx + 1) * self.batch_size]
        # create empty array to hold images
        # batch.sort()
        x = np.empty((self.batch_size, 224, 224, 1))
        y = np.zeros((self.batch_size, 2))
        times_gen = []
        # loop through batch
      
        for i, img in enumerate(batch):
            # populate time and position labels 
            y[i] = extract_position_time(img)[0]
            time = extract_position_time(img)[1]
            times_gen.append(normalize_datetime(time, times.min(), times.max()))
             # read image
            img = cv2.imread(os.path.join(self.directory, img),0)
            img = cv2.resize(img, (224, 224))
            # add image to x
            x[i] = img.reshape(224, 224, 1)

        # get output
        output = normalize_y(y, position)
           
        # get time input
        time_input = np.array(times_gen) 
        
        return [x/255, time_input], output

custom_gen_train = CustomGenerator(directory = '../images/train', segment = 'Train')
custom_gen_val = CustomGenerator(directory = '../images/valid', segment = 'Validate')

In [8]:
# import efficientnet
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D

# import resnet
from tensorflow.keras.applications import ResNet50



# define the input layers
input_image = Input(shape=(224,224,1))
input_time = Input(shape=times[0].shape)

# convolutional layer that feeds into efficientnet
x = Conv2D(3,(3,3),padding='same')(input_image) 
# process the input image using efficientnet
resnet = ResNet50(weights='imagenet',include_top= 'TRUE') 

x = resnet(x)


# reshape input_time 
t = tf.keras.layers.Reshape(target_shape=(1, 1))(input_time)

# process the input time using LSTM layers
t = LSTM(32, return_sequences=True)(t)
t = LSTM(16)(t)

# concatenate the output of the CNN and LSTM layers
z = tf.keras.layers.concatenate([x, t], axis=-1)

# z = concatenate([x, t])
z = Dense(256, activation='relu')(z)
z = Dropout(0.2)(z)
outputs = Dense(2, activation='sigmoid')(z)

# define the model
model = Model(inputs=[input_image, input_time], outputs=outputs)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_8 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_7 (InputLayer)           [(None, 224, 224, 1  0           []                               
                                )]                                                                
                                                                                                  
 reshape_1 (Reshape)            (None, 1, 1)         0           ['input_8[0][0]']                
                                                                                                  
 conv2d_2 (Conv2D)              (None, 224, 224, 3)  30          ['input_7[0][0]']          

In [22]:
# fit the model
model.compile(optimizer='adam', loss=haversine_loss, metrics=['mse'])
history = model.fit(custom_gen_train, validation_data=custom_gen_val, epochs=10, verbose=1)

Epoch 1/10
   2/2222 [..............................] - ETA: 8:31:52 - loss: 53.6193 - mse: 0.1101 

KeyboardInterrupt: 